# Statistics Final Project Machine Learning Model

In [2]:
import pandas as pd
# Imports
source = lambda x : f'../star-categorization-giants-and-dwarfs/{x}'

X_full = pd.read_csv(source('Star39552_balanced.csv'))
X_full.head()

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,10.00,31.66,6.19,1.213,K7V,22.502556,1
1,8.26,3.21,1.00,1.130,K0III,15.792525,0
2,8.27,12.75,1.06,0.596,F9V,18.797552,1
3,6.54,5.23,0.76,1.189,K1III,15.132508,0
4,8.52,0.96,0.72,0.173,B8V,13.431356,1


Selecting target and predictor variables to be 'Amag' and 'B-V'

In [3]:
from sklearn.model_selection import train_test_split

X_cols = ['Amag', 'B-V']
y_cols = ['TargetClass']

# print(list(set(X_full.columns) - set(X_cols + y_cols)))

X_full.dropna(axis=0, subset=['TargetClass'], inplace=True)
y = X_full[y_cols]
X_full.drop(list(set(X_full.columns) - set(X_cols)), axis=1, inplace=True)

X_train_full, X_valid_full, y_train, y_full = train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state = 100)

In [4]:
print(X_train_full.head())
print(X_valid_full.head())
print(X_train_full.shape)
print(X_valid_full.shape)

         B-V       Amag
34566  1.535  14.652069
36393  0.526  13.954068
22237  1.079  14.161129
34775  0.962  15.951990
28850  0.066  12.376615
         B-V       Amag
2438   0.331  17.863170
20570  0.481  18.138397
18388  1.563  13.793582
11136  1.747  15.034814
37525  1.352  14.257706
(31641, 2)
(7911, 2)


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(n_estimators=1, random_state=0)
model.fit(X_train_full, y_train)
y_pred = model.predict(X_valid_full)
print(f"MSE: {mean_squared_error(y_pred, y_full)}")

MSE: 0.18897737327771458


/tmp/ipykernel_38748/2554294135.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_full, y_train)


In [6]:
print(y_pred)

[1. 1. 0. ... 0. 0. 1.]


In [7]:
print(y_full)

       TargetClass
2438             0
20570            1
18388            0
11136            0
37525            0
...            ...
4977             0
376              1
15970            0
35839            1
23676            0

[7911 rows x 1 columns]


In [9]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_full, y_pred, labels=[0, 1])

array([[3202,  769],
       [ 726, 3214]])